# Transaction Data Collection from the Ethereum Blockchain

## EIP-1559
- Date: August 5, 2021
- Block number: 12,965,000
- [Ethereum JSON-RPC Specification](https://ethereum.github.io/execution-apis/api-documentation/)
- [JSON RPC API](https://ethereum.org/en/developers/docs/apis/json-rpc/)
- [EIP-1559 Analysis Arxiv](https://github.com/SciEcon/EIP1559)

## Layer 2 Solutions Launch Dates
Source: [L2BEAT](https://l2beat.com/scaling/tvl)
1. Optimism is live on: January 16, 2021
2. Arbitrum is live on: August 31, 2021

In [1]:
import datetime
import csv
import os
import glob
from datetime import datetime
import time

import pandas as pd
from collections import defaultdict

In [4]:
def timestamp(date_string):
    """
    Convert a date string to a Unix timestamp.
    
    Args:
        date_string (str): The date string in 'YYYY-MM-DD' format.
    
    Returns:
        int: The Unix timestamp corresponding to the date string.
    """
    dt = datetime.strptime(date_string, "%Y-%m-%d")
    return int(dt.timestamp())


### Merge the data

In [ ]:
# Set the directory you want to start from
rootDir = '../data/'  # current directory; adjust it if needed

# Find all CSV files in the directory
all_files = glob.glob(os.path.join(rootDir, "eth_transaction_data_*.csv"))

# Read and concatenate all files into one DataFrame
merged_data = pd.concat((pd.read_csv(file) for file in all_files))

# Save the combined data to a new CSV file
merged_data.to_csv('merged_eth_transaction_data.csv', index=False)


In [ ]:
def get_unique_senders(data):
    """
    Collect all unique sender addresses from a dataframe of transaction data.
    
    Args:
        data (DataFrame): A dataframe representing transaction data.
    
    Returns:
        DataFrame: A dataframe of unique sender addresses.
    """
    unique_senders = data['Sender\'s Address'].unique()
    return pd.DataFrame(unique_senders, columns=['Sender\'s Address'])

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('merged_eth_transaction_data.csv')
df.columns

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace("'", "")

In [ ]:
# Get unique sender addresses
unique_senders = get_unique_senders(df)

In [ ]:
# If you want to save the unique senders to a CSV file
unique_senders.to_csv('unique_senders.csv', index=False)

In [ ]:
# Convert Unix timestamp to pandas Timestamp
df['transaction_timestamp'] = pd.to_datetime(df['transaction_timestamp'], unit='s')

# Sort by user address and timestamp
df.sort_values(['senders_address', 'transaction_timestamp'], inplace=True)

# Add a column for transaction count within the last 12 hours, initialized with 0
df['trans_freq'] = 0

# Initialize the dictionary to store transaction counts
transaction_counts = defaultdict(list)

# Loop over each row in the DataFrame
for i in range(len(df)):
    # Get the current user and transaction timestamp
    current_user = df.iloc[i]['senders_address']
    current_time = df.iloc[i]['transaction_timestamp']

    # Define the 12-hour window start time
    window_start = current_time - pd.Timedelta(hours=12)

    # Store transaction timestamps for each user
    transaction_counts[current_user].append(current_time)

    # Keep only transactions within the 12-hour window
    transaction_counts[current_user] = [ts for ts in transaction_counts[current_user] if ts >= window_start]

    # Set the transaction frequency for the current row to the number of transactions within the time window
    df.at[i, 'trans_freq'] = len(transaction_counts[current_user])


In [ ]:
df[:10]

In [ ]:
# Convert string to datetime and create a temporary column
df['temp_timestamp'] = pd.to_datetime(df['transaction_timestamp'])

# Set the launch dates for Layer 2 solutions
optimism_launch = datetime(2021, 1, 16)
arbitrum_launch = datetime(2021, 8, 31)

# Add the 'layer2_availability' column
df['layer2_availability'] = ((df['temp_timestamp'] >= arbitrum_launch) | (df['temp_timestamp'] >= optimism_launch)).astype(int)

# Add the 'post_eip1559' column. 
# EIP-1559 went live on August 5, 2021
eip1559_launch_date = datetime(2021, 8, 5)
df['post_eip1559'] = (df['temp_timestamp'] >= eip1559_launch_date).astype(int)

# Drop the temporary column
df = df.drop('temp_timestamp', axis=1)

In [ ]:
df[:10]

---
1. **User transaction count**: The cumulative number of transactions made by the same user up to the current transaction.
2. **Failed transactions**: the cumulative number of failed transactions by the same user up to the current transaction

In [ ]:
# Create a new column that is 1 if the transaction failed and 0 otherwise
df['is_failed'] = (df['transaction_status'] == 'False').astype(int)

# Group by user address and calculate the cumulative count of transactions and failed transactions
df = df.sort_values('transaction_timestamp')
df['user_transaction_count'] = df.groupby('senders_address').cumcount() + 1
df['failed_transactions'] = df.groupby('senders_address')['is_failed'].cumsum()

# You can drop the 'is_failed' column if you no longer need it
df = df.drop('is_failed', axis=1)
df[:10]

In this script, we first created a new column `is_failed` that is 1 if the transaction failed and 0 otherwise. We then sorted the dataframe by the transaction timestamp to ensure transactions are processed in the order they occurred. 

Next, we grouped the dataframe by the sender's address and used `cumcount` to get the cumulative count of transactions for each user (we added 1 because `cumcount` starts from 0). We also used `cumsum` on the `is_failed` column to get the cumulative count of failed transactions for each user. 

Finally, we dropped the `is_failed` column as it's no longer needed.

---

In [ ]:
df.columns

---
Derive User Address Age

We are using 'eth.getTransactionCount' function from web3.py, which retrieves the number of transactions sent from an address up until a certain block. Using this function, we can find the block at which the address was first used.


In the script below, for each transaction in the dataframe, the script scans blocks from the genesis block to the block of the transaction. If the address has made any transactions up to a block, it means that this block is the first usage block of the address, so it's saved to the `first_usage` dictionary. The `get_transaction_count` function is used to get the number of transactions sent from an address up to a certain block. After finding the first usage block of each address, the script adds a new column 'address_age' to the data, which represents the age of the address at the time of each transaction.

The above solution is likely to be slow if you have a large number of unique addresses and transactions. It's because for each address, it scans blocks from the genesis block to the block of each transaction of the address. If you have a large number of unique addresses, the total number of blocks to scan can be enormous.

To optimize the calculation of user address age and utilize a high-performance computing cluster, we could use parallel computing techniques. This involves dividing the computation task into smaller jobs that can be run simultaneously across multiple processors or nodes in the cluster.

In Python, several libraries allow you to use parallel computing, such as `multiprocessing`, `concurrent.futures`, and `joblib`. Here's an example of how you could use the `concurrent.futures` library to parallelize the block scanning task. This script uses a `ThreadPoolExecutor` to run multiple block scanning tasks simultaneously, which should significantly speed up the process if you're running it on a machine with multiple CPU cores. However, the maximum number of concurrent tasks is limited by the number of CPU cores in your machine.

In [ ]:
# Function to find the first usage block of an address
def find_first_usage(address, transaction_block):
    # Scan blocks from the start block to the transaction block
    for block in range(start_block, transaction_block):
        # If the address has made any transactions up to the block
        if web3.eth.get_transaction_count(address, block):
            return block
    return None

In [ ]:
# Sort by block number
df.sort_values(['block_number'], inplace=True)

# Prepare a dictionary to store the first usage block of each address
first_usage = {}

# Define the range of blocks to scan
start_block = 0  # This should be set to the genesis block

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # For each unique address in the data
    for address in df['senders_address'].unique():
        transaction_block = df[df['senders_address'] == address]['block_number'].min()
        
        # If the first usage block of the address has already been found, skip this address
        if address in first_usage:
            continue
        
        # Submit a new task to the executor
        future = executor.submit(find_first_usage, address, transaction_block)
        
        # Store the Future object in the dictionary
        first_usage[address] = future

# Retrieve the results from the Future objects
for address, future in first_usage.items():
    first_usage[address] = future.result()

# Add a new column 'address_age' to the data
df['address_age'] = df['senders_address'].map(first_usage)

df[:10]

---

**Time of the day variable**

Due to the global nature of Ethereum transactions, the "time of day" variable could have different implications for users in different time zones, and this could indeed introduce complexity and potentially confounding effects into the analysis.

However, there may still be value in including a "time of day" variable, even in a global context, for several reasons:

1. **Network Effects**: Blockchain networks like Ethereum may experience periods of higher and lower congestion, which could align with certain times of day, despite global usage. For example, if a substantial proportion of Ethereum users are based in a particular region (say, North America or East Asia), then the network might be busier during the waking hours of that region.

2. **Market Activity**: Cryptocurrency markets operate 24/7 and market activity (trading volume, price volatility, etc.) can vary significantly across different times of the day, potentially influencing user behavior. For example, users might be more likely to engage in DeFi transactions during periods of high market activity.

3. **Behavioral Patterns**: Regardless of the global nature of Ethereum, there might be common daily behavioral patterns. For example, users might be more active during their daytime and less active during their nighttime, and these patterns could aggregate up to observable patterns in the data.

However, given the global nature, it may be advisable to construct the "time of day" variable in a way that captures potential global effects. For example, you could split the day into fewer, larger chunks (like morning, afternoon, evening, and night), or analyze this variable carefully in your exploratory analysis to understand its distribution and potential impacts.

Another approach might be to construct a variable that captures the "local time of day" for each transaction, assuming you can infer or have information about the geographic location of each user (which could introduce privacy issues and may not be feasible). But again, these are more complex and may not necessarily offer a better representation.

Finally, including "time of day" in your initial model does not obligate you to keep it in your final model. If exploratory analysis or preliminary model results suggest it's not meaningful or is causing issues, you can always exclude it in later iterations of your modeling process.

In conclusion, the "time of day" could potentially be a meaningful variable in your analysis, but its use and interpretation require careful consideration due to the global nature of Ethereum usage.

Given the global nature of Ethereum, a simple division of a day into distinct periods based on a specific time zone may not be the most representative. However, a potential solution could be to divide the day into a number of periods that are likely to capture significant changes in activity. For instance:

1. **Daytime**: 06:00 to 17:59
2. **Evening**: 18:00 to 21:59
3. **Night**: 22:00 to 05:59

This division attempts to capture typical working hours (daytime), after-work hours (evening), and sleeping hours (night). Of course, given the global nature of the network, these periods won't align perfectly with these times for all users, but they might serve as a useful approximation.

If it's possible to incorporate additional information, such as the geographic distribution of Ethereum users or the times of day that tend to see the most network activity, this could be used to refine these periods further.

You can then create a new variable, "TimeOfDay", in your dataset by mapping each transaction timestamp to one of these periods.

Here's a Python script to do that assuming your timestamp is in the form 'YYYY-MM-DD HH:MM:SS' and in UTC:

In [ ]:
def assign_time_of_day(timestamp):
    hour = timestamp.hour
    if 6 <= hour < 18:
        return 'Daytime'
    elif 18 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

df['time_of_day'] = df['transaction_timestamp'].apply(assign_time_of_day)